<a href="https://www.kaggle.com/code/aleksandrmorozov123/nlp-with-transformers?scriptVersionId=196773253" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/llm-fine-tuning-dataset-of-indian-legal-texts/constitution_qa.json
/kaggle/input/llm-fine-tuning-dataset-of-indian-legal-texts/ipc_qa.json
/kaggle/input/llm-fine-tuning-dataset-of-indian-legal-texts/crpc_qa.json
/kaggle/input/wine-reviews/winemag-data_first150k.csv
/kaggle/input/wine-reviews/winemag-data-130k-v2.json
/kaggle/input/wine-reviews/winemag-data-130k-v2.csv
/kaggle/input/vivino-brazilian-wines-metadata/brazilian_wine_data.json
/kaggle/input/vivino-brazilian-wines-metadata/brazilian_wine_data.csv
/kaggle/input/spanish-wine-quality-dataset/wines_SPA.csv
/kaggle/input/pakistan-law-data/pdf_data.json
/kaggle/input/sec-edgar-annual-financial-filings-2021/extracted/1378590_10K_2021_0001437749-21-028984.json
/kaggle/input/sec-edgar-annual-financial-filings-2021/extracted/1408278_10K_2020_0001408278-21-000050.json
/kaggle/input/sec-edgar-annual-financial-filings-2021/extracted/1066194_10K_2021_0001558370-21-012388.json
/kaggle/input/sec-edgar-annual-financial-filings-2

In [2]:
import numpy as np
import json
import re
import string
from IPython.display import display, HTML
import pandas as pd

import tensorflow as tf
from tensorflow.keras import layers, models, losses, callbacks

2024-09-15 16:04:21.532086: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-09-15 16:04:21.532196: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-09-15 16:04:21.839803: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


**Set the parameters of the model**

In [3]:
VOCAB_SIZE = 10000
MAX_LEN = 80
EMBEDDING_DIM = 256
KEY_DIM = 256
N_HEADS = 2
FEED_FORWARD_DIM = 256
VALIDATION_SPLIT = 0.2
SEED = 42
LOAD_MODEL = False
BATCH_SIZE = 32
EPOCHS = 5

**Loading the data**

In [4]:
# Load the full dataset
with open("/kaggle/input/wine-reviews/winemag-data-130k-v2.json") as json_data:
    wine_data = json.load(json_data)

In [5]:
# Filter the dataset
filtered_data = [
    "wine review : "
    + x["title"]
    + " : "
    + x["description"]
    + " : "
    + x["variety"]
    + " : "
    + x["province"]
    for x in wine_data
    if x ["title"] is not None
    and x["description"] is not None
    and x["variety"] is not None
    and x["province"] is not None
]

In [6]:
# count the reviews
n_wines = len (filtered_data)
print (f"{n_wines} reviews loaded")

129907 reviews loaded


In [7]:
example = filtered_data [20]
print (example)

wine review : Quiévremont 2012 Vin de Maison Red (Virginia) : Ripe aromas of dark berries mingle with ample notes of black pepper, toasted vanilla and dusty tobacco. The palate is oak-driven in nature, but notes of tart red currant shine through, offering a bit of levity. : Red Blend : Virginia


**Tokenize the data**

In [8]:
# Pad the punctuation, to treat them as separate 'words'
def pad_punctuation(s):
    s = re.sub(f"([{string.punctuation}, '\n'])", r" \1 ", s)
    s = re.sub(" +", " ", s)
    return s


text_data = [pad_punctuation(x) for x in filtered_data]

In [9]:
# example of the data
example_data = text_data[20]
example_data

'wine review : Quiévremont 2012 Vin de Maison Red ( Virginia ) : Ripe aromas of dark berries mingle with ample notes of black pepper , toasted vanilla and dusty tobacco . The palate is oak - driven in nature , but notes of tart red currant shine through , offering a bit of levity . : Red Blend : Virginia'

In [10]:
# Convert to a Tensorflow Dataset
text_ds = (
    tf.data.Dataset.from_tensor_slices(text_data)
    .batch(BATCH_SIZE)
    .shuffle(1000)
)

In [11]:
# Create a vectorisation layer
vectorize_layer = layers.TextVectorization(
    standardize="lower",
    max_tokens=VOCAB_SIZE,
    output_mode="int",
    output_sequence_length=MAX_LEN + 1,
)

In [12]:
# Adapt the layer to the training set
vectorize_layer.adapt(text_ds)
vocab = vectorize_layer.get_vocabulary()

In [13]:
# Display some token:word mappings
for i, word in enumerate(vocab[:10]):
    print(f"{i}: {word}")

0: 
1: [UNK]
2: :
3: ,
4: .
5: and
6: the
7: wine
8: a
9: of


**Create the training set**

In [14]:
# Create the training set and the same text shifted by one word
def prepare_inputs(text):
    text = tf.expand_dims(text, -1)
    tokenized_sentences = vectorize_layer(text)
    x = tokenized_sentences[:, :-1]
    y = tokenized_sentences[:, 1:]
    return x, y


train_ds = text_ds.map(prepare_inputs)

In [15]:
example_input_output = train_ds.take(1).get_single_element()

In [16]:
# Example Input
example_input_output[0][0]

<tf.Tensor: shape=(80,), dtype=int64, numpy=
array([   7,   10,    2,    1,   69,    1,   55,  935, 2500,   31,   43,
         12,  772,   11,    2,    8,    7,  138,  123,   14,  603,   18,
        256,  278,  190, 1569, 1861,    4,   17,  504,  469,   52,  935,
         15, 2500,  581,   13,  250, 1122,    5,  334,   20,  142,  979,
        989,    4,   59,   62,    3, 3517,    5,  220,  980,  407,   17,
        714,   49,    8, 1350, 5390,   26,    1, 1140,  123, 1769, 4226,
          3,    1, 1727,  128,  687,    1,    4,   17,   47,   71,  321,
          5,  464,   28])>

In [17]:
# Example Output (shifted by one token)
example_input_output[1][0]

<tf.Tensor: shape=(80,), dtype=int64, numpy=
array([  10,    2,    1,   69,    1,   55,  935, 2500,   31,   43,   12,
        772,   11,    2,    8,    7,  138,  123,   14,  603,   18,  256,
        278,  190, 1569, 1861,    4,   17,  504,  469,   52,  935,   15,
       2500,  581,   13,  250, 1122,    5,  334,   20,  142,  979,  989,
          4,   59,   62,    3, 3517,    5,  220,  980,  407,   17,  714,
         49,    8, 1350, 5390,   26,    1, 1140,  123, 1769, 4226,    3,
          1, 1727,  128,  687,    1,    4,   17,   47,   71,  321,    5,
        464,   28,    9])>

**Casual attention mask function**

In [18]:
def causal_attention_mask(batch_size, n_dest, n_src, dtype):
    i = tf.range(n_dest)[:, None]
    j = tf.range(n_src)
    m = i >= j - n_src + n_dest
    mask = tf.cast(m, dtype)
    mask = tf.reshape(mask, [1, n_dest, n_src])
    mult = tf.concat(
        [tf.expand_dims(batch_size, -1), tf.constant([1, 1], dtype=tf.int32)], 0
    )
    return tf.tile(mask, mult)


np.transpose(causal_attention_mask(1, 10, 10, dtype=tf.int32)[0])

array([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
       [0, 1, 1, 1, 1, 1, 1, 1, 1, 1],
       [0, 0, 1, 1, 1, 1, 1, 1, 1, 1],
       [0, 0, 0, 1, 1, 1, 1, 1, 1, 1],
       [0, 0, 0, 0, 1, 1, 1, 1, 1, 1],
       [0, 0, 0, 0, 0, 1, 1, 1, 1, 1],
       [0, 0, 0, 0, 0, 0, 1, 1, 1, 1],
       [0, 0, 0, 0, 0, 0, 0, 1, 1, 1],
       [0, 0, 0, 0, 0, 0, 0, 0, 1, 1],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 1]], dtype=int32)

**Transformer block layer**

In [19]:
class TransformerBlock(layers.Layer):
    def __init__(self, num_heads, key_dim, embed_dim, ff_dim, dropout_rate=0.1):
        super(TransformerBlock, self).__init__()
        self.num_heads = num_heads
        self.key_dim = key_dim
        self.embed_dim = embed_dim
        self.ff_dim = ff_dim
        self.dropout_rate = dropout_rate
        self.attn = layers.MultiHeadAttention(
            num_heads, key_dim, output_shape=embed_dim
        )
        self.dropout_1 = layers.Dropout(self.dropout_rate)
        self.ln_1 = layers.LayerNormalization(epsilon=1e-6)
        self.ffn_1 = layers.Dense(self.ff_dim, activation="relu")
        self.ffn_2 = layers.Dense(self.embed_dim)
        self.dropout_2 = layers.Dropout(self.dropout_rate)
        self.ln_2 = layers.LayerNormalization(epsilon=1e-6)

    def call(self, inputs):
        input_shape = tf.shape(inputs)
        batch_size = input_shape[0]
        seq_len = input_shape[1]
        causal_mask = causal_attention_mask(
            batch_size, seq_len, seq_len, tf.bool
        )
        attention_output, attention_scores = self.attn(
            inputs,
            inputs,
            attention_mask=causal_mask,
            return_attention_scores=True,
        )
        attention_output = self.dropout_1(attention_output)
        out1 = self.ln_1(inputs + attention_output)
        ffn_1 = self.ffn_1(out1)
        ffn_2 = self.ffn_2(ffn_1)
        ffn_output = self.dropout_2(ffn_2)
        return (self.ln_2(out1 + ffn_output), attention_scores)

    def get_config(self):
        config = super().get_config()
        config.update(
            {
                "key_dim": self.key_dim,
                "embed_dim": self.embed_dim,
                "num_heads": self.num_heads,
                "ff_dim": self.ff_dim,
                "dropout_rate": self.dropout_rate,
            }
        )
        return config

**Token and Position embedding**

In [20]:
class TokenAndPositionEmbedding(layers.Layer):
    def __init__(self, max_len, vocab_size, embed_dim):
        super(TokenAndPositionEmbedding, self).__init__()
        self.max_len = max_len
        self.vocab_size = vocab_size
        self.embed_dim = embed_dim
        self.token_emb = layers.Embedding(
            input_dim=vocab_size, output_dim=embed_dim
        )
        self.pos_emb = layers.Embedding(input_dim=max_len, output_dim=embed_dim)

    def call(self, x):
        maxlen = tf.shape(x)[-1]
        positions = tf.range(start=0, limit=maxlen, delta=1)
        positions = self.pos_emb(positions)
        x = self.token_emb(x)
        return x + positions

    def get_config(self):
        config = super().get_config()
        config.update(
            {
                "max_len": self.max_len,
                "vocab_size": self.vocab_size,
                "embed_dim": self.embed_dim,
            }
        )
        return config

**Build the Ttansformer model**

In [21]:
inputs = layers.Input(shape=(None,), dtype=tf.int32)
x = TokenAndPositionEmbedding(MAX_LEN, VOCAB_SIZE, EMBEDDING_DIM)(inputs)
x, attention_scores = TransformerBlock(
    N_HEADS, KEY_DIM, EMBEDDING_DIM, FEED_FORWARD_DIM
)(x)
outputs = layers.Dense(VOCAB_SIZE, activation="softmax")(x)
gpt = models.Model(inputs=inputs, outputs=[outputs, attention_scores])
gpt.compile("adam", loss=[losses.SparseCategoricalCrossentropy(), None])

In [22]:
gpt.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, None)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ token_and_position_embedding    │ (None, None, 256)      │     2,580,480 │
│ (TokenAndPositionEmbedding)     │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ transformer_block               │ [(None, None, 256),    │       658,688 │
│ (TransformerBlock)              │ (None, 2, None, None)] │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, None, 10000)    │     2,570,000 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 5,809,168 (22.16 MB)

 Trainable params: 5,809,168 (22.16 MB)

 Non-trainable params: 0 (0.00 B)

**Train the Transformer model**

In [23]:
# Create a TextGenerator checkpoint
class TextGenerator(callbacks.Callback):
    def __init__(self, index_to_word, top_k=10):
        self.index_to_word = index_to_word
        self.word_to_index = {
            word: index for index, word in enumerate(index_to_word)
        }

    def sample_from(self, probs, temperature):
        probs = probs ** (1 / temperature)
        probs = probs / np.sum(probs)
        return np.random.choice(len(probs), p=probs), probs

    def generate(self, start_prompt, max_tokens, temperature):
        start_tokens = [
            self.word_to_index.get(x, 1) for x in start_prompt.split()
        ]
        sample_token = None
        info = []
        while len(start_tokens) < max_tokens and sample_token != 0:
            x = np.array([start_tokens])
            y, att = self.model.predict(x, verbose=0)
            sample_token, probs = self.sample_from(y[0][-1], temperature)
            info.append(
                {
                    "prompt": start_prompt,
                    "word_probs": probs,
                    "atts": att[0, :, -1, :],
                }
            )
            start_tokens.append(sample_token)
            start_prompt = start_prompt + " " + self.index_to_word[sample_token]
        print(f"\ngenerated text:\n{start_prompt}\n")
        return info

    def on_epoch_end(self, epoch, logs=None):
        self.generate("wine review", max_tokens=80, temperature=1.0)

In [24]:
# Create a model save checkpoint
model_checkpoint_callback = callbacks.ModelCheckpoint(
    filepath="./checkpoint/checkpoint.ckpt.weights.h5",
    save_weights_only=True,
    save_freq="epoch",
    verbose=0,
)

tensorboard_callback = callbacks.TensorBoard(log_dir="./logs")

# Tokenize starting prompt
text_generator = TextGenerator(vocab)

In [25]:
gpt.fit(
    train_ds,
    epochs=EPOCHS,
    callbacks=[model_checkpoint_callback, tensorboard_callback, text_generator],
)

Epoch 1/5
   3/4060 ━━━━━━━━━━━━━━━━━━━━ 2:05 31ms/step - loss: 8.9943   

I0000 00:00:1726416335.910131      72 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.
W0000 00:00:1726416335.932760      72 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


3250/4060 ━━━━━━━━━━━━━━━━━━━━ 17s 22ms/step - loss: 3.0352

W0000 00:00:1726416407.343058      72 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


4060/4060 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - loss: 2.9372

W0000 00:00:1726416456.142330      70 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update
W0000 00:00:1726416467.559767      71 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update



generated text:
wine review : [UNK] 2015 [UNK] adler mondeuse red ( costières de nîmes ) : the first release from this new - equal bottle , wine is one of the better known for reds despite their southern oregon wineries , namely all five years . this wine opens with classic barossa valley now , serve it ' s born jardin , with [UNK] flavors of lusty , cabernet franc and black pepper . the huge concentration and strength it finishes

4060/4060 ━━━━━━━━━━━━━━━━━━━━ 151s 35ms/step - loss: 2.9371
Epoch 2/5
4060/4060 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - loss: 2.1776
generated text:
wine review : [UNK] 2015 grüner veltliner ( austria ) : rounder and rich , this has weight of the wine ' s ripest apples and a toasty aroma . the palate is apple while still aging adds a richer , style of chardonnay . the tannins are just right by the rené there is citrus and a delicious edge , creating the wine that lasts long into the spicy side . pair with rich texture and concentration . :

4060/4060 ━━━━━━━━━

**Generate text using the Transformer**

In [26]:
def print_probs(info, vocab, top_k=5):
    for i in info:
        highlighted_text = []
        for word, att_score in zip(
            i["prompt"].split(), np.mean(i["atts"], axis=0)
        ):
            highlighted_text.append(
                '<span style="background-color:rgba(135,206,250,'
                + str(att_score / max(np.mean(i["atts"], axis=0)))
                + ');">'
                + word
                + "</span>"
            )
        highlighted_text = " ".join(highlighted_text)
        display(HTML(highlighted_text))

        word_probs = i["word_probs"]
        p_sorted = np.sort(word_probs)[::-1][:top_k]
        i_sorted = np.argsort(word_probs)[::-1][:top_k]
        for p, i in zip(p_sorted, i_sorted):
            print(f"{vocab[i]}:   \t{np.round(100*p,2)}%")
        print("--------\n")

In [27]:
info = text_generator.generate(
    "wine review: Toscana", max_tokens=80, temperature=1.0
)


generated text:
wine review: Toscana imperial ( prosecco ( prosecco ) ) : this historic winery is among its mouthwatering expressions of nebbiolo . overt floral , fragrant expressions of gewürztraminer , [UNK] and fully - ripened grapes for a wine . : glera : veneto 



In [28]:
info = text_generator.generate(
    "wine review: Toscana", max_tokens=80, temperature=0.5
)


generated text:
wine review: Toscana [UNK] ( [UNK] ) : [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] , ” is [UNK] ' [UNK] - [UNK] ' [UNK] - [UNK] grape [UNK] . the [UNK] is [UNK] , the [UNK] [UNK] that [UNK] the [UNK] [UNK] of [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] . ” [UNK] , ” and it ' s a big , sticky wine , with ripe , jammy fruit and vanilla . : [UNK] : [UNK] 



In [29]:
info = text_generator.generate(
    "wine review: Toscana", max_tokens=80, temperature=0.5
)
print_probs(info, vocab)


generated text:
wine review: Toscana [UNK] ( barbera 2007 ) : this is a [UNK] with a simple , jammy , jammy berry and cherry flavors . the wine is soft and easy to drink , with a soft , gentle fade . : barbera : piedmont 



[UNK]:   	99.02%
(:   	0.36%
trocken:   	0.11%
red:   	0.05%
vineyard:   	0.05%
--------



(:   	91.87%
[UNK]:   	1.88%
riserva:   	1.62%
riesling:   	1.47%
white:   	0.72%
--------



[UNK]:   	44.74%
barbaresco:   	19.22%
barbera:   	9.15%
valpolicella:   	7.31%
taurasi:   	5.62%
--------



2007:   	25.08%
di:   	15.69%
del:   	11.81%
2012:   	8.77%
2010:   	8.61%
--------



...

::   	98.2%
it:   	0.71%
the:   	0.68%
drink:   	0.32%
a:   	0.03%
--------



barbera:   	99.99%
moscato:   	0.01%
merlot:   	0.01%
dolcetto:   	0.0%
primitivo:   	0.0%
--------



::   	100.0%
d:   	0.0%
[UNK]:   	0.0%
':   	0.0%
,:   	0.0%
--------



piedmont:   	95.12%
california:   	4.71%
monferrato:   	0.13%
sicily:   	0.02%
barbera:   	0.01%
--------



:   	100.0%
::   	0.0%
red:   	0.0%
[UNK]:   	0.0%
other:   	0.0%
--------

